###**THIS IS A CODE TO CREATE CSV FOR TARGET2 FOR MEDIS ATTACK.**

**TARGET2:** HIGHEST *MEANIOU* BASED TARGET.

**OUTPUT:** A csv file contains *nxn* entries; *n* is total number of images for each dataset. Each entry depicts the meanIOU between row image to column image.

**NOTE:** we set the diagonal entries as -1. (Target should not be similar to the actual image for adversarial attack)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **PraNet (Polyp)**

In [ ]:
import os
os.chdir('/content/drive/MyDrive/PraNet/')

In [ ]:
# IMPORT ALL REQUIRED LIBRARIES
import os
import numpy as np
from utils.dataloader import test_dataset,transforms
from PIL import Image
import torch
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix
from csv import DictReader

In [ ]:
# SET DEVICE
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [ ]:
# FUNCTION TO LOAD DATA AND COMPUTE METRIC
def mask_loader(path,testdata):
   with open(path, 'rb') as f:
    img = Image.open(f)
    mask = img.convert('L')
    if testdata == 'Kvasir':
      resize = transforms.Resize((500,540))
      mask = resize(mask) # for kvasir data only
    mask = np.asarray(mask, np.float32)
    mask /= (mask.max() + 1e-8)
    mask = torch.from_numpy(mask).to(device)
    return mask

def compute_iou(prediction, target):
  y_pred = prediction.flatten()
  y_true = target.flatten()
  current = confusion_matrix(y_true.round(), y_pred.round(), labels=[0, 1])
  intersection = np.diag(current)
  ground_truth_set = current.sum(axis=1)
  predicted_set = current.sum(axis=0)
  union = ground_truth_set + predicted_set - intersection
  IoU = intersection / union.astype(np.float32)
  MeanIoU = np.nanmean(IoU)
  return round(MeanIoU,3)

In [ ]:
# MAIN FILE FOR POLYP
for testdata in tqdm(['CVC-300', 'CVC-ClinicDB','CVC-ColonDB', 'Kvasir']):
  path = 'TestDataset/'+testdata+'/masks/' # path to dataset
  data = os.listdir(path) # a list of all filename in dataset
  d = {}
  header = ['Base Image']
  masks = []
  for i in tqdm(data): # loop over all data for row
    header.append(str(i))
    value = []
    mask1 = mask_loader(path+str(i),testdata)
    for j in data: # loop over all data for column
      if i == j:
        value.append(-1)
        continue
      mask2 = mask_loader(path+str(j),testdata)
      mIoU = round(torch.tensor([compute_iou(mask1.cpu().numpy(),mask2.cpu().numpy())]).item(),4)
      value.append(mIoU)
    d.update({i:value})
  filename = '/content/drive/MyDrive/PraNet/code_check_medis/' + str(testdata) + '_target2.csv' # csv file name
  pd.DataFrame(d).T.reset_index().to_csv(filename, header=header, index=False)
print('Done!')

# **Skin Lesion (Melanoma)**

In [ ]:
# MAIN FILE FOR SKIN LESION
path = 'melanomaData/testy/' # path to dataset
data = os.listdir(path) # a list of all filename in dataset
d = {}
header = ['Base Image']
masks = []
for i in tqdm(data):
  header.append(str(i))
  value = []
  mask1 = mask_loader(path+str(i),None)
  for j in data:
    if i == j:
      value.append(-1)
      continue
    mask2 = mask_loader(path+str(j),None)
    mIoU = round(torch.tensor([compute_iou(mask1.detach().cpu().numpy(),mask2.detach().cpu().numpy())]).item(),4)
    value.append(mIoU)
  d.update({i:value})
filename = '/content/drive/MyDrive/PraNet/code_check_medis/skin_lesion_target2.csv' # csv file name
pd.DataFrame(d).T.reset_index().to_csv(filename, header=header, index=False)
print('Done!')